In [ ]:
from dask import delayed, compute
import dask.dataframe as dd

import pandas as pd
import numpy as np

import time

In [ ]:
def make_data(size: float=1e8) -> pd.DataFrame:
    size = int(size)
    df = pd.DataFrame(dict(
        a=np.random.rand(size),
        b=np.random.choice(list('wxyz'), size=size),
        c=np.random.randint(10, size=size)
    ), columns=list('abc'))
    for label in ['b']:
        df[label] = df[label].astype('category')
    return df

In [ ]:
%%time
data = make_data(1e8)

In [ ]:
data.memory_usage()

In [ ]:
def work1(df: pd.DataFrame) -> pd.DataFrame:
    time.sleep(1)
    return data.groupby(data.b).sum()

def work2(df: pd.DataFrame) -> pd.DataFrame:
    time.sleep(2)
    return data.groupby(data.c).mean()

In [ ]:
%%time
work1(data);
work2(data);

In [ ]:
import multiprocessing as mp

In [ ]:
p1 = mp.Process(target=work1, args=(data,))
p2 = mp.Process(target=work2, args=(data,))

In [ ]:
%%time
p1.start()
p2.start()
p1.join()
p2.join()